In [ ]:
import pandas as pd
from geopy.distance import geodesic  # pip install geopy 

# Leer CSV clientes con lat y lon
df_clientes = pd.read_csv('clientes_90min.csv')
df_clientes = df_clientes[df_clientes["CLIENTE"].apply(lambda x: str(x).isdigit())]
df_clientes["CLIENTE"] = df_clientes["CLIENTE"].astype(int)

# Crear matriz vacía
clientes = df_clientes["CLIENTE"].tolist()
n = len(clientes)

# Diccionario para tiempos (minutos)
velocidad_kmh = 50  # velocidad media supuesta
matriz_tiempos = pd.DataFrame(index=clientes, columns=clientes, dtype=float)

for c1 in clientes:
    coord1 = df_clientes.loc[df_clientes["CLIENTE"]==c1, ["lat","lon"]].values[0]
    for c2 in clientes:
        coord2 = df_clientes.loc[df_clientes["CLIENTE"]==c2, ["lat","lon"]].values[0]
        distancia_km = geodesic(coord1, coord2).km
        tiempo_min = (distancia_km / velocidad_kmh) * 60
        matriz_tiempos.loc[c1, c2] = tiempo_min

print(matriz_tiempos)


           1           3          4          5          6           7   \
1    0.000000   20.179006  47.635422   0.126675  40.379977   82.670932   
3   20.179006    0.000000  67.814338  20.287950  60.558861  100.418250   
4   47.635422   67.814338   0.000000  47.526394   7.263546   49.081834   
5    0.126675   20.287950  47.526394   0.000000  40.271466   82.612238   
6   40.379977   60.558861   7.263546  40.271466   0.000000   52.562097   
7   82.670932  100.418250  49.081834  82.612238  52.562097    0.000000   
9   43.410773   49.699618  61.214412  43.464317  56.064673   66.003625   
10  10.147997   25.336241  45.435237  10.175102  38.295917   75.136752   
11  81.054481   98.958293  46.968773  80.993860  50.514425    2.146484   
17   0.928277   20.916485  46.907256   0.802828  39.657020   82.354626   
18  33.183914   52.606663  19.854625  33.059512  14.698931   67.235858   
19  33.769360   49.383272  36.016018  33.649180  31.566446   84.127579   
20  18.168896   32.910831  42.159717  

In [5]:
import pandas as pd
import openrouteservice
from itertools import product
import time  # 👈 Añadido para poder hacer pausas

# Parámetros
CSV_PATH = 'clientes_90min.csv'
OUTPUT_PARES = 'matriz_tiempos_pares_openroute.csv'
ORS_API_KEY = 'eyJvcmciOiI1YjNjZTM1OTc4NTExMTAwMDFjZjYyNDgiLCJpZCI6ImJhZjE1ZWE5ZDZlMzQ5MThhOGE4MmJmMDQ2NzUxMTFjIiwiaCI6Im11cm11cjY0In0'

# Cargar datos
df_clientes = pd.read_csv(CSV_PATH)
df_clientes.columns = [col.strip().upper() for col in df_clientes.columns]
df_clientes = df_clientes[['CLIENTE', 'LAT', 'LON']]
df_clientes.dropna(subset=['LAT', 'LON'], inplace=True)

# Combinaciones de pares únicas (excluye cliente consigo mismo)
pares = [
    (origen['CLIENTE'], destino['CLIENTE'], [origen['LON'], origen['LAT']], [destino['LON'], destino['LAT']])
    for origen, destino in product(df_clientes.to_dict('records'), repeat=2)
    if origen['CLIENTE'] != destino['CLIENTE']
]

# Calcular tiempos con OpenRouteService
client = openrouteservice.Client(key="eyJvcmciOiI1YjNjZTM1OTc4NTExMTAwMDFjZjYyNDgiLCJpZCI6ImJhZjE1ZWE5ZDZlMzQ5MThhOGE4MmJmMDQ2NzUxMTFjIiwiaCI6Im11cm11cjY0In0=") 
filas = []

for origen_id, destino_id, coord_origen, coord_destino in pares:
    try:
        route = client.directions(
            coordinates=[coord_origen, coord_destino],
            profile='driving-car',
            format='geojson'
        )
        duracion_min = route['features'][0]['properties']['segments'][0]['duration'] / 60
        filas.append({
            'ORIGEN': origen_id,
            'DESTINO': destino_id,
            'TIEMPO_MIN': round(duracion_min, 2)
        })
        time.sleep(1.5)  # ⏱️ Pausa para no saturar la API
    except Exception as e:
        print(f"⚠️ Error entre {origen_id} ➡ {destino_id}: {e}")

# 💾 Guardar resultados
df_resultados = pd.DataFrame(filas)
df_resultados.to_csv(OUTPUT_PARES, index=False)
print(f"✅ Matriz de tiempos por pares guardada en: {OUTPUT_PARES}")


c:\Users\34666\AppData\Local\Programs\Python\Python311\Lib\site-packages\openrouteservice\client.py:211: UserWarning: Rate limit exceeded. Retrying for the 1st time.
  warnings.warn('Rate limit exceeded. Retrying for the {0}{1} time.'.format(retry_counter + 1,
c:\Users\34666\AppData\Local\Programs\Python\Python311\Lib\site-packages\openrouteservice\client.py:211: UserWarning: Rate limit exceeded. Retrying for the 2nd time.
  warnings.warn('Rate limit exceeded. Retrying for the {0}{1} time.'.format(retry_counter + 1,
c:\Users\34666\AppData\Local\Programs\Python\Python311\Lib\site-packages\openrouteservice\client.py:211: UserWarning: Rate limit exceeded. Retrying for the 3rd time.
  warnings.warn('Rate limit exceeded. Retrying for the {0}{1} time.'.format(retry_counter + 1,
c:\Users\34666\AppData\Local\Programs\Python\Python311\Lib\site-packages\openrouteservice\client.py:211: UserWarning: Rate limit exceeded. Retrying for the 4th time.
  warnings.warn('Rate limit exceeded. Retrying for 

KeyboardInterrupt: 